In [ ]:
!pip install optimum[exporters-tf]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.0/301.0 kB 5.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 77.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 43.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 585.9/585.9 MB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.7/454.7 kB 35.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 85.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 84.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 86.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 

In [ ]:
!optimum-cli export tflite --help

2023-10-28 18:25:08.952193: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
usage: optimum-cli export tflite [-h] -m MODEL [--task TASK] [--atol ATOL]
                                 [--pad_token_id PAD_TOKEN_ID] [--cache_dir CACHE_DIR]
                                 [--trust-remote-code] [--batch_size BATCH_SIZE]
                                 [--sequence_length SEQUENCE_LENGTH] [--num_choices NUM_CHOICES]
                                 [--width WIDTH] [--height HEIGHT] [--num_channels NUM_CHANNELS]
                                 [--feature_size FEATURE_SIZE] [--nb_max_frames NB_MAX_FRAMES]
                                 [--audio_sequence_length AUDIO_SEQUENCE_LENGTH]
                                 [--quantize {int8-dynamic,int8,int8x16,fp16}]
                                 [--fallback_to_float] [--inputs_type {int8,uint8}]
                                 [--outputs_type {int8,uint8}]
                                 [--c

In [ ]:
import torch
import json
import numpy as np
import pandas as pd
import transformers
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5TokenizerFast
from torch.utils.data import Dataset, DataLoader, RandomSampler
from tqdm import tqdm
from torch.optim import Adam

TOKENZIER=T5TokenizerFast.from_pretrained('t5-base')
MODEL=T5ForConditionalGeneration.from_pretrained('t5-base', return_dict=True)
OPTIMIZER=Adam(MODEL.parameters(), lr=0.00001)
Q_LEN=256
T_LEN=64
BATCH_SIZE=4
DEVICE=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class Medical(Dataset):
    def __init__(self,tokenizer, data, q_len, t_len):
        self.tokenizer=tokenizer
        self.data=data
        self.q_len=q_len
        self.t_len=t_len
        self.question=self.data['question']
        self.context=self.data['context']
        self.answer=self.data['answer']

    def __len__(self):
        return len(self.question)

    def __getitem__(self, index):
        question=self.question[index]
        context=self.context[index]
        answer=self.answer[index]

        source=self.tokenizer(question, context, max_length=self.q_len, padding="max_length", truncation=True, pad_to_max_length=True, add_special_tokens=True)
        target=self.tokenizer(answer, max_length=self.t_len, padding="max_length", truncation=True, pad_to_max_length=True, add_special_tokens=True)

        labels=torch.tensor(target['input_ids'], dtype=torch.long)
        labels[labels==0]=-100

        return {
            "input_ids": torch.tensor(source['input_ids'], dtype=torch.long),
            "attention_mask": torch.tensor(source['attention_mask'], dtype=torch.long),
            "labels": labels,
            "decoder_attention_mask": torch.tensor(target['attention_mask'], dtype=torch.long)
        }


def get_data(data):
    articles=[]

    for article in data["data"]:
        for paragraph in article["paragraphs"]:
            for qa in paragraph["qas"]:
                question=qa["question"]
                context=paragraph["context"]
                if qa["is_impossible"]:
                    answer="no answer"
                else:
                    answer=qa["answers"][0]["text"]
                inputs={"context": context, "question": question, "answer": answer}
                articles.append(inputs)
    return articles



with open('test_webmd_squad_v2_full.json', 'r') as f:
    train_data = json.load(f)

data=get_data(train_data)

data=pd.DataFrame(data)

with open('test_webmd_squad_v2_consec.json', 'r') as f:
    test_data = json.load(f)

test_data=get_data(test_data)

test_data=pd.DataFrame(test_data)

# DataLoaders

train_sampler=RandomSampler(data.index)
test_sampler=RandomSampler(test_data.index)

medcal_dataset=Medical(TOKENZIER, data, Q_LEN, T_LEN)

train_dataloader=DataLoader(medcal_dataset, sampler=train_sampler, batch_size=BATCH_SIZE)
test_dataloader=DataLoader(medcal_dataset, sampler=test_sampler, batch_size=BATCH_SIZE)


# Training

train_loss=0
val_loss=0
train_batch=0
val_batch=0

for epoch in range(2):
    print(f"Epoch {epoch+1}/{2}")
    print('-'*10)
    MODEL.train()

    for batch in tqdm(train_dataloader,desc="Training batches"):
        input_ids=batch["input_ids"].to(DEVICE)
        attention_mask=batch["attention_mask"].to(DEVICE)
        labels=batch["labels"].to(DEVICE)
        decoder_attention_mask=batch["decoder_attention_mask"].to(DEVICE)

        OPTIMIZER.zero_grad()

        outputs=MODEL(input_ids=input_ids, attention_mask=attention_mask, labels=labels, decoder_attention_mask=decoder_attention_mask)

        loss=outputs.loss
        loss.backward()
        OPTIMIZER.step()
        train_loss+=loss.item()

        train_batch+=1

    print(f"Train loss: {train_loss/train_batch}")

    MODEL.eval()

    for batch in tqdm(test_dataloader,desc="Validation batches"):
        input_ids=batch["input_ids"].to(DEVICE)
        attention_mask=batch["attention_mask"].to(DEVICE)
        labels=batch["labels"].to(DEVICE)
        decoder_attention_mask=batch["decoder_attention_mask"].to(DEVICE)

        outputs=MODEL(input_ids=input_ids, attention_mask=attention_mask, labels=labels, decoder_attention_mask=decoder_attention_mask)

        OPTIMIZER.zero_grad()
        loss=outputs.loss
        loss.backward()
        OPTIMIZER.step()
        val_loss+=loss.item()
        val_batch+=1

    print(f"Validation loss: {val_loss/val_batch}")


# Save model
MODEL.save_pretrained("qa_model")
TOKENZIER.save_pretrained("qa_model_tokenizer")


/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5_fast.py:158: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


FileNotFoundError: ignored

In [ ]:
!optimum-cli export tflite --model t5-base --sequence_length 128 t5_tflite/

2023-10-28 18:59:20.604927: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-10-28 18:59:26.910027: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.
Traceback (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.10/dist-packages/optimum/exporters/tflite/__main__.py", line 146, in <module>
    main()
  File "/usr/local/lib/python3.10/dist-packa